# Create rolling CF% graphs for individuals
Game plan: read game by game stats, set parameters, calculate, plot.

### Import

In [1]:
import PbPMethods2 as pm2
from pylab import *
%matplotlib inline
import pandas as pd
import GetPbP
import matplotlib.patheffects as path_effects

### Generate game-by-game data 
Do this only if you haven't already at least once. (It saves to file, so you don't need to run it every time you open this notebook.) Re-run to update current season data. (The '#' ensures code after it on the same line doesn't run.)
<p>This will take a long time and give you some errors--known errors are <a href="https://github.com/muneebalam/Hockey/blob/master/NHL/Quick-graphs/gengamebygameerrors.txt">listed here</a>.

In [2]:
seasons_to_update = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
#pm2.gen_gamebygame(season)

### Read in game by game data
Do this only once, each time you open this notebook, to load data into memory.

In [3]:
dfs = []
for season in range(2007, 2017):
    dfs.append(pd.read_csv(pm2.get_gamebygame_data_filename(season)))
dfs = pd.concat(dfs)
#dfs.head()

### Select the player, teams, seasons etc you want.
If you want to generate more graphs start re-running code from here.

In [100]:
player = "Andre Burakovsky"
start_date = '2012-09-15' #use mid-Sept to be safe because some seasons have Euro premiere games
end_date = '2017-06-21' #Format: yyyy-mm-dd

teams = 'all' #or, e.g. {'WSH'}, or {'WSH', 'T.B'}
roll_len = 25 #in games

score_adjusted = True #Can also set to False. Method conceptually similar to @IneffectiveMath
include_playoffs = True #can also set to False
save_file = None #set to something to save figure
gapless = True #Will avoid long offseason flatlines (no games = no change in CF%)
    
if teams == 'all':
    teams = GetPbP.TEAMS

### Filter based on above parameters
Note how I select a date range here. This can be very useful (for example, if you want to edit the usage charts notebook code to take a date range instead of specific seasons).

In [101]:
df = dfs[dfs.Player == player]
df = df[df.Team.apply(lambda x: x in teams)]

df['Date2'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
df.sort_values('Date2', inplace=True)
df.set_index('Date2', inplace=True)
df = df.ix[start_date : end_date]

if gapless:
    df.reset_index(inplace=True)

if not include_playoffs:
    df = df[df.Game <= 21230]

### Calculate

In [102]:
if score_adjusted:
    cols_to_use = ['SACF', 'SACA']
else:
    cols_to_use = ['CF', 'CA']
    
end_vars = [cols_to_use[0]+'%', cols_to_use[0]+' Off%']
cols_to_use = [cols_to_use[0]+'ON', cols_to_use[1]+'ON', 
               cols_to_use[0]+'OFF', cols_to_use[1]+'OFF']

In [103]:
cols_to_use2 = []
for col in cols_to_use:
    cols_to_use2.append('{0:d}-game {1:s}'.format(roll_len, col))
    df[cols_to_use2[-1]] = pd.rolling_mean(df[col], roll_len, 0)
df[end_vars[0]] = df[cols_to_use2[0]] / (df[cols_to_use2[0]] + df[cols_to_use2[1]])
df[end_vars[1]] = df[cols_to_use2[2]] / (df[cols_to_use2[2]] + df[cols_to_use2[3]])

In [104]:
def format_timestamp_as_mdyy(ts):
    st = str(ts).split(' ')[0]
    y, m, d = st.split('-')
    m = int(m)
    d = int(d)
    return '{0:d}/{1:d}\n{2:s}'.format(m, d, y)

In [105]:
indices_to_change_colors = [0]
teamnames = [df['Team'].iloc[0]]
team_annot_x = []
team_annot_y = [0.37]
colors = GetPbP.TEAM_COLOR_DCT[teamnames[0]]
c1 = colors[0]
c2 = colors[1]
color1 = [c1]
color2 = [c2]
next_year_indices = [0]
start_date2 = pd.Timestamp(df['Date2'].iloc[0])
end_date2 = pd.Timestamp(df['Date2'].iloc[-1])
next_year_dates = [format_timestamp_as_mdyy(start_date2)]
years_elapsed = 0
oneyr = pd.Timedelta('365 days')
for i in range(1, len(df)):
    if gapless:
        current_date = df['Date2'].iloc[i]
    else:
        current_date = df.index[i]
    if current_date - start_date2 >= (years_elapsed + 1) * oneyr:
        next_year_indices.append(i)
        next_year_dates.append(format_timestamp_as_mdyy(current_date))
        years_elapsed += 1
    
    if df['Team'].iloc[i] == df['Team'].iloc[i - 1]:
        pass
    else:
        indices_to_change_colors.append(i)
        teamnames.append(df['Team'].iloc[i])
        c1, c2 = GetPbP.TEAM_COLOR_DCT[teamnames[-1]]
        color1.append(c1)
        color2.append(c2)
        
        dt = df.index[indices_to_change_colors[-1] - 1]
        dt = dt - df.index[indices_to_change_colors[-2]]
        
        team_annot_x.append(df.index[indices_to_change_colors[-2]] + dt/2)
        nexty = team_annot_y[0] - team_annot_y[-1] + 0.4
        team_annot_y.append(nexty) #alternate between 0.37 and 0.40
        
indices_to_change_colors.append(len(df))
dt = df.index[-1]
dt = dt - df.index[indices_to_change_colors[-2]]
team_annot_x.append(df.index[indices_to_change_colors[-2]] + dt/2)

### Plot

In [106]:
for i in range(len(indices_to_change_colors) - 1):
    starti = indices_to_change_colors[i]
    endi = indices_to_change_colors[i + 1]
    
    #plot dual-color line
    line = plot(df[end_vars[0]].iloc[starti : endi + 1], c=color2[i], 
                lw=1, label='CF% On')
    line[0].set_path_effects([path_effects.Stroke(linewidth=3, foreground=color1[i]),
                           path_effects.Normal()])
    
    line = plot(df[end_vars[1]].iloc[starti : endi + 1], c=color2[i], 
                lw=1, label='CF% Off', ls='--', alpha=0.5)
    line[0].set_path_effects([path_effects.Stroke(linewidth=2, foreground=color1[i]),
                           path_effects.Normal()])
    
    annotate(teamnames[i], xy=(team_annot_x[i], team_annot_y[i]), va='bottom',
            bbox=dict(boxstyle='round', facecolor='wheat', edgecolor='black'), fontsize=16)
    
    if i == 0:
        legend(loc=1, title='Legend', fontsize=10, ncol=2, framealpha=0.5)
    
ylim(0.35, 0.65)
ys = arange(0.35, 0.70, 0.05)
yticks(ys, ['{0:.0f}%'.format(y*100) for y in ys])

if gapless:
    xlim(0, len(df))
    xticks(next_year_indices, next_year_dates, ha='left')
else:
    xlim(start_date2, end_date2)
if score_adjusted:
    titletext = '{0:d}-game rolling SACF% for {1:s}'.format(roll_len, player)
else:
    titletext = '{0:d}-game rolling CF% for {1:s}'.format(roll_len, player)
titletext += '\n{0:s} to {1:s}'.format(format_timestamp_as_mdyy(start_date2).replace('\n', '/'), 
                                       format_timestamp_as_mdyy(end_date2).replace('\n', '/'))
title(titletext)

if save_file is not None:
    savefig(save_file)